# MOE Trading System Demo

This notebook demonstrates how to use the MOE Trading System.

## Setup

First, make sure you've installed all required packages and set up your API keys in `config.json`.

In [ ]:
# Import required modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from datetime import datetime, timedelta

# Import MOE system components
from expert import Expert
from strategies import TrendFollowingExpert, MeanReversionExpert, VolatilityExpert, SentimentExpert
from meeting import ExpertMeeting
from backtester import Backtester
from main import run_moe_system

# Verify API keys
try:
    with open('config.json', 'r') as f:
        config = json.load(f)
        openai_key = config.get('openai_api_key')
        gemini_key = config.get('gemini_api_key')
        
    if openai_key == "your_openai_api_key_here" or gemini_key == "your_gemini_api_key_here":
        print("⚠️ Please update your API keys in config.json")
    else:
        print("✅ API keys found in config.json")
except Exception as e:
    print(f"❌ Error loading config.json: {e}")


## Run the MOE System

Let's run the MOE system with 2 rounds and 2 meeting rounds per expert meeting.

In [ ]:
# Run with smaller parameters for faster execution
results = run_moe_system(
    n_rounds=2,
    k_meeting_rounds=2,
    test_duration_days=10,
    training_duration_days=30
)

print("\nResults Summary:")
print(f"Round performances: {[f'{p:.2%}' for p in results['round_performances']]}")
print(f"Average performance: {results['average_performance']:.2%}")
print(f"Best performance: {results['best_performance']:.2%} (Round {results['best_round']})")

## Compare with S&P 500

Now let's compare our MOE system performance with the S&P 500.

In [ ]:
import yfinance as yf

def get_spy_returns(start_date, end_date):
    try:
        spy_data = yf.download("SPY", start=start_date, end=end_date)
        if len(spy_data) < 2:
            return 0.0
        first_price = float(spy_data['Close'].iloc[0])
        last_price = float(spy_data['Close'].iloc[-1])
        return (last_price / first_price) - 1
    except Exception as e:
        print(f"Error getting SPY data: {e}")
        return 0.0

# Load backtest results to get date ranges
spy_returns = []
for i in range(len(results['round_performances'])):
    try:
        with open(f"backtest_results_round{i+1}.json", "r") as f:
            backtest = json.load(f)
            start_date = backtest.get("start_date")
            end_date = backtest.get("end_date")
            if start_date and end_date:
                spy_return = get_spy_returns(start_date, end_date)
                spy_returns.append(spy_return)
    except Exception as e:
        print(f"Error loading backtest results: {e}")

# Create comparison table
comparison = pd.DataFrame({
    'Round': range(1, len(results['round_performances']) + 1),
    'MOE Performance': [f"{p:.2%}" for p in results['round_performances']],
    'S&P 500 Performance': [f"{p:.2%}" for p in spy_returns],
    'Difference': [f"{results['round_performances'][i] - spy_returns[i]:.2%}" for i in range(len(spy_returns))]
})

# Add average row
comparison.loc[len(comparison)] = [
    'Average',
    f"{sum(results['round_performances']) / len(results['round_performances']):.2%}",
    f"{sum(spy_returns) / len(spy_returns):.2%}",
    f"{sum(results['round_performances']) / len(results['round_performances']) - sum(spy_returns) / len(spy_returns):.2%}"
]

comparison

## Visualize Performance

Let's create a simple visualization of our MOE system performance compared to S&P 500.

In [ ]:
plt.figure(figsize=(12, 6))

# Plot round performances
rounds = list(range(1, len(results['round_performances']) + 1))
plt.bar([r - 0.2 for r in rounds], [p * 100 for p in results['round_performances']], width=0.4, label='MOE Strategy', color='blue')
plt.bar([r + 0.2 for r in rounds], [p * 100 for p in spy_returns], width=0.4, label='S&P 500', color='green')

plt.xlabel('Round')
plt.ylabel('Return (%)')
plt.title('MOE vs S&P 500 Performance by Round')
plt.xticks(rounds)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Expert Meeting Analysis

Let's examine one of the expert meeting logs to see how the experts collaborated.

In [ ]:
# Find the most recent meeting log
import glob

meeting_logs = sorted(glob.glob("meeting_log_*.json"), key=os.path.getmtime, reverse=True)
if meeting_logs:
    with open(meeting_logs[0], 'r') as f:
        meeting = json.load(f)
    
    # Print first round discussion
    if meeting and len(meeting) > 0 and 'discussion' in meeting[0]:
        print(f"Meeting log: {meeting_logs[0]}")
        print("\nFirst round discussion excerpt:")
        discussion = meeting[0]['discussion']
        # Print first 500 characters
        print(discussion[:500] + "...\n")
        
        # Print consensus
        if 'consensus' in meeting[-1]:
            consensus = meeting[-1]['consensus']
            print("Final Consensus:")
            print(f"Action: {consensus.get('action')}")
            print(f"Confidence: {consensus.get('confidence'):.2f}")
            print(f"Supporting experts: {', '.join(consensus.get('supporting_experts', []))}")
            print(f"Vote ratio: {consensus.get('vote_ratio', 0):.2f}")
else:
    print("No meeting logs found.")

## Conclusion

This demo shows how the MOE Trading System works:

1. Multiple experts analyze the market data independently
2. They meet, discuss, and try to reach consensus
3. The consensus decision is tested against historical market data
4. Experts learn from their performance and improve over time

The system's performance can be compared with benchmark indices like the S&P 500 to evaluate its effectiveness.